In [1]:
from __future__ import division
import argparse
import torch
import torch.nn.functional as F
import torchvision.datasets as dset
import torchvision.transforms as T
import torchvision.models as models
import pandas as pd
import os
import pydicom
import numpy as np
import h5py
import time
from tqdm import tqdm as tqdm
import cv2
import pickle
from shutil import copyfile
from matplotlib import pyplot as plt
import seaborn as sns
import random
from torch import nn
from torch.autograd import Variable

Image Level
===========

In [2]:
class StudyLevelDataset(torch.utils.data.Dataset):
    """Kaggle PE dataset."""

    def __init__(self, csv_file, stage):
        """
        Args:
            csv_file (string): Path to the csv file.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.pedataframe = pd.read_csv(csv_file)
        
        self.hdf5_filename = '/scratch/features.hdf5'
        
        self._generate_row_indices_list()
        
        self.stage = stage
        
        if self.stage == 'train':
            self.start = 0
            self.end = 2650
        else:
            self.start = 2660
            self.end = 3500
        
        # 0 - 2650 train
        # 2660 - 3500 valid

    def __len__(self):
        """ Return number of 2D images. (Each CT slice is an independent image.)"""
        #return len(self.pedataframe)
        return self.end - self.start + 1
        #return num_pos_samples_per_dataset + num_neg_samples_per_dataset
        
    def _generate_row_indices_list(self):
        # group slice indices into studies
        self.row_indices = [] # index into train df for every study
        current_study_id = ''

        for slice_index in range(len(self.pedataframe)):
            new_study_id = self.pedataframe.StudyInstanceUID[slice_index]
            if new_study_id != current_study_id:
                self.row_indices.append(slice_index)
                current_study_id = new_study_id
            if slice_index % 100000 == 0:
                print(slice_index)
        
    def __getitem__(self, study_index):
        '''  '''
        study_index = study_index + self.start
        
        h5py_file = h5py.File(self.hdf5_filename, "r")
        
        start_index = self.row_indices[study_index]
        if study_index+1 < len(self.row_indices):
            stop_index = self.row_indices[study_index+1]
        else:
            stop_index = len(self.pedataframe)
            
        # dim: seq_len x embedding_dim
        x = np.ones((stop_index-start_index, 2048))
        y = np.ones(stop_index-start_index)
            
        for slice_index in range(start_index, stop_index):
            data_identifier = 'tensor(' + str(slice_index) + ')'
            slice_embeddings = h5py_file[data_identifier][:]
            x[slice_index-start_index,:] = slice_embeddings
            
            pe_present_on_image = float(int(self.pedataframe.pe_present_on_image[slice_index]))
            y[slice_index-start_index] = pe_present_on_image
        
        h5py_file.close()
        
        return x, y

In [3]:
hdf5_filename = '/scratch/features.hdf5'
h5py_file = h5py.File(hdf5_filename, "r")
h5py_file['tensor(12)'][:]

h5py_file.close()

In [4]:
data_dir = '/projectnb/ece601/kaggle-pulmonary-embolism/rsna-str-pulmonary-embolism-detection/'
train_csv = data_dir + 'train.csv'
train_dir = data_dir + 'train/'
train_dataset = StudyLevelDataset(csv_file=train_csv, stage='train')
valid_dataset = StudyLevelDataset(csv_file=train_csv, stage='valid')

0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000


In [5]:
'''
https://towardsdatascience.com/taming-lstms-variable-sized-mini-batches-and-why-pytorch-is-good-for-your-health-61d35642972e
'''

batch_size = 1
embedding_dim = 2048
nb_lstm_units = 2048

class MyLSTM(nn.Module):
    def __init__(self, nb_layers=1, nb_lstm_units=nb_lstm_units, 
                 embedding_dim=embedding_dim, batch_size=batch_size):
        
        super(MyLSTM, self).__init__()

        self.nb_layers = nb_layers
        self.nb_lstm_units = nb_lstm_units
        self.embedding_dim = embedding_dim
        self.batch_size = batch_size

        self.lstm = nn.LSTM(
            input_size=self.embedding_dim,
            hidden_size=self.nb_lstm_units,
            num_layers=self.nb_layers,
            batch_first=True
        )

        # output layer
        self.linear = nn.Linear(self.nb_lstm_units, 1)

    def init_hidden(self):
        # the weights are of the form (nb_layers, batch_size, nb_lstm_units)
        hidden_h0 = torch.randn(self.nb_layers, self.batch_size, self.nb_lstm_units)
        hidden_c0 = torch.randn(self.nb_layers, self.batch_size, self.nb_lstm_units)

        hidden_h0 = hidden_h0.to(device)
        hidden_c0 = hidden_c0.to(device)

        hidden_h0 = Variable(hidden_h0)
        hidden_c0 = Variable(hidden_c0)

        return (hidden_h0.to(device), hidden_c0.to(device))
    
    def forward(self, X):
        # reset the LSTM hidden state. Must be done before you run a new batch. Otherwise the LSTM will treat
        # a new batch as a continuation of a sequence
        self.hidden = self.init_hidden()

        batch_size, seq_len, _ = X.size()

        # pack_padded_sequence so that padded items in the sequence won't be shown to the LSTM
        #X = torch.nn.utils.rnn.pack_padded_sequence(x, X_lengths, batch_first=True)

        # now run through LSTM
        X, self.hidden = self.lstm(X, self.hidden)

        # undo the packing operation
        #X, _ = torch.nn.utils.rnn.pad_packed_sequence(X, batch_first=True)

        # this one is a bit tricky as well. First we need to reshape the data so it goes into the linear layer
        X = X.contiguous()
        X = X.view(-1, X.shape[2])

        # run through actual linear layer
        X = self.linear(X)

        # ---------------------
        # 4. Create softmax activations bc we're doing classification
        # Dim transformation: (batch_size * seq_len, nb_lstm_units) -> (batch_size, seq_len, nb_tags)
        #X = torch.sigmoid(X)

        # I like to reshape for mental sanity so we're back to (batch_size, seq_len, nb_tags)
        X = X.view(batch_size, seq_len)

        Y_hat = X
        return Y_hat

In [6]:
model = MyLSTM()
model

MyLSTM(
  (lstm): LSTM(2048, 2048, batch_first=True)
  (linear): Linear(in_features=2048, out_features=1, bias=True)
)

In [154]:
device = 'cpu'
model.to(device)
x,y = dataset[1]
x = torch.tensor(x)
x = x.unsqueeze(0).float()
x = x.to(device)

pred = model(x)

In [33]:
bce = torch.nn.BCEWithLogitsLoss()
def criterion(logits, target):
    loss = bce(logits.view(-1), target.view(-1))
    return loss

dummy = None

def train_epoch(model, loader, optimizer):

    model.train()
    train_loss = []
    bar = tqdm(loader)
    for (data, target) in bar:
        data, target = data.to(device), target.to(device)
        
        # zero gradients
        optimizer.zero_grad()

        #forward pass
        logits = model(data.float())
        
        # calculate loss
        loss = criterion(logits, target)

        # backpropagate the loss (backward pass)
        loss.backward()
 
        # update parameters based on accumulated gradients
        optimizer.step()

        loss_np = loss.detach().cpu().numpy()

        train_loss.append(loss_np)
        average_loss = sum(train_loss) / len(train_loss)
        bar.set_description('loss: %.5f, mean: %.5f' % (loss_np, average_loss))

    return float(average_loss)

def valid_epoch(model, loader):
    model.eval()
    val_loss = []
    pos_logits = []
    neg_logits = []

    with torch.no_grad():
        for (data, target) in tqdm(loader):
            data, target = data.to(device), target.to(device)
            logits = model(data.float())

            loss = criterion(logits, target)

            loss_np = loss.detach().cpu().numpy()

            val_loss.append(float(loss_np))

            logits = logits.squeeze()
            target = target.squeeze()
            
            for i in range(logits.shape[-1]):
                b = target[i].detach().cpu().numpy()
                if b == 1:
                    pos_logits.append(float(logits[i].detach().cpu().numpy()))
                else:
                    neg_logits.append(float(logits[i].detach().cpu().numpy()))
                    
    val_loss_mean = sum(val_loss) / len(val_loss)
    
    neg_logits_tensor = torch.FloatTensor(neg_logits).cuda()
    pos_logits_tensor = torch.FloatTensor(pos_logits).cuda()
    
    neg_loss = criterion(neg_logits_tensor, torch.zeros(neg_logits_tensor.shape).float().cuda())
    pos_loss = criterion(pos_logits_tensor, torch.ones(pos_logits_tensor.shape).float().cuda())
    
    neg_loss = neg_loss.detach().cpu().numpy()
    pos_loss = pos_loss.detach().cpu().numpy()
    
    neg_mean = float(torch.sigmoid(neg_logits_tensor).mean().detach().cpu().numpy())
    pos_mean = float(torch.sigmoid(pos_logits_tensor).mean().detach().cpu().numpy())
    
    return float(val_loss_mean), float(pos_loss), float(neg_loss), pos_mean, neg_mean

def get_optimizer(lr, model):
    optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=0.0001)
    return optimizer

init_lr = 0.1
n_epochs = 20
device = 'cuda'

# reduce LR by gamma three times every 10 epochs
# each epoch is 100,000 samples
#gamma = 10
#schedule = [10, 20, 30]

#model = resnext101.to(device)
optimizer = get_optimizer(init_lr, model)
model.to(device)

master_train_loss = []
master_valid_loss = []
epoch = 1
#best_valid_loss = 10


while epoch <= n_epochs:
    print(time.ctime(), 'Epoch:', epoch)
    
    # update learning rate 
    #if epoch in schedule:
    #    new_lr = optimizer.param_groups[0]["lr"] / gamma
     #   optimizer = get_optimizer(new_lr, model)
    
    #train_loader, valid_loader = get_loaders(epoch)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=1,num_workers=1)
    valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=1,num_workers=1)

    # train
    train_loss = train_epoch(model, train_loader, optimizer)
    optimizer.zero_grad()
    
    # validate
    valid_loss, pos_loss, neg_loss, pos_mean, neg_mean = valid_epoch(model, valid_loader)
    #valid_loss = 0
    #pos_loss  = 0
    #neg_loss = 0
    #pos_mean = 0
    #neg_mean = 0
    
    content = time.ctime() + ' ' + f'Epoch {epoch}, lr: {optimizer.param_groups[0]["lr"]:.7f}, train loss: {np.mean(train_loss):.5f}, valid loss: {(valid_loss):.5f}'
    print(content)
    content = f'pos loss: {(pos_loss):.5f}, neg loss: {(neg_loss):.5f}, pos mean: {(pos_mean):.5f}, neg mean {(neg_mean):.5f}'
    print(content)
    master_train_loss.append(train_loss)
    master_valid_loss.append(valid_loss)
    
    # save loss data and model weights
    #with open('train_loss.pkl', 'wb') as f:
        #pickle.dump(master_train_loss, f)
    #with open('valid_loss.pkl', 'wb') as f:
        #pickle.dump(master_valid_loss, f)
        
        #torch.save(model.state_dict(), 'model-resnext-50-{}.pth'.format(epoch))
        #best_valid_loss = valid_loss
    
    epoch += 1
           

  0%|          | 0/2651 [00:00<?, ?it/s]

Wed Dec  2 22:03:49 2020 Epoch: 1


  0%|          | 0/2651 [00:00<?, ?it/s]

Wed Dec  2 22:08:06 2020 Epoch 1, lr: 0.1000000, train loss: 0.11742, valid loss: 0.11713
pos loss: 1.06458, neg loss: 0.06358, pos mean: 0.47428, neg mean 0.05167
Wed Dec  2 22:08:06 2020 Epoch: 2


  0%|          | 0/2651 [00:00<?, ?it/s]

Wed Dec  2 22:12:15 2020 Epoch 2, lr: 0.1000000, train loss: 0.11430, valid loss: 0.11619
pos loss: 1.02854, neg loss: 0.06458, pos mean: 0.48526, neg mean 0.05239
Wed Dec  2 22:12:15 2020 Epoch: 3


loss: 0.02645, mean: 0.15282:  11%|█         | 296/2651 [00:18<02:30, 15.63it/s]


KeyboardInterrupt: 

Study Level Metrics
===============

In [58]:
class StudyLevelDataset2(torch.utils.data.Dataset):
    """Kaggle PE dataset."""

    def __init__(self, csv_file, stage):
        """
        Args:
            csv_file (string): Path to the csv file.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.pedataframe = pd.read_csv(csv_file)
        
        self.hdf5_filename = '/scratch/features.hdf5'
        
        self._generate_row_indices_list()
        
        self.stage = stage
        
        if self.stage == 'train':
            self.start = 0
            self.end = 2650
        else:
            self.start = 2660
            self.end = 3500
        
        # 0 - 2650 train
        # 2660 - 3500 valid

    def __len__(self):
        """ Return number of 2D images. (Each CT slice is an independent image.)"""
        #return len(self.pedataframe)
        return self.end - self.start + 1
        #return num_pos_samples_per_dataset + num_neg_samples_per_dataset
        
    def _generate_row_indices_list(self):
        # group slice indices into studies
        self.row_indices = [] # index into train df for every study
        current_study_id = ''

        for slice_index in range(len(self.pedataframe)):
            new_study_id = self.pedataframe.StudyInstanceUID[slice_index]
            if new_study_id != current_study_id:
                self.row_indices.append(slice_index)
                current_study_id = new_study_id
            if slice_index % 100000 == 0:
                print(slice_index)
        
    def __getitem__(self, study_index):
        '''  '''
        study_index = study_index + self.start
        
        h5py_file = h5py.File(self.hdf5_filename, "r")
        
        start_index = self.row_indices[study_index]
        if study_index+1 < len(self.row_indices):
            stop_index = self.row_indices[study_index+1]
        else:
            stop_index = len(self.pedataframe)
            
        # dim: seq_len x embedding_dim
        x = np.ones((stop_index-start_index, 2048))
            
        for slice_index in range(start_index, stop_index):
            data_identifier = 'tensor(' + str(slice_index) + ')'
            slice_embeddings = h5py_file[data_identifier][:]
            x[slice_index-start_index,:] = slice_embeddings

            
        y = np.ones(9)
            
        y[0] =  self.pedataframe.negative_exam_for_pe[start_index]
        y[1] = self.pedataframe.rv_lv_ratio_gte_1[start_index]
        y[2] = self.pedataframe.rv_lv_ratio_lt_1[start_index]
        y[3] = self.pedataframe.leftsided_pe[start_index]
        y[4] = self.pedataframe.chronic_pe[start_index]
        y[5] = self.pedataframe.rightsided_pe[start_index]
        y[6] = self.pedataframe.acute_and_chronic_pe[start_index]
        y[7] = self.pedataframe.central_pe[start_index]
        y[8] = self.pedataframe.indeterminate[start_index]
        
        h5py_file.close()
        
        return x, y

In [59]:
data_dir = '/projectnb/ece601/kaggle-pulmonary-embolism/rsna-str-pulmonary-embolism-detection/'
train_csv = data_dir + 'train.csv'
train_dir = data_dir + 'train/'
train_dataset = StudyLevelDataset2(csv_file=train_csv, stage='train')
valid_dataset = StudyLevelDataset2(csv_file=train_csv, stage='valid')

0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000


In [60]:
'''
https://towardsdatascience.com/taming-lstms-variable-sized-mini-batches-and-why-pytorch-is-good-for-your-health-61d35642972e
'''

batch_size = 1
embedding_dim = 2048
nb_lstm_units = 2048

class StudyLevelLSTM(nn.Module):
    def __init__(self, nb_layers=1, nb_lstm_units=nb_lstm_units, 
                 embedding_dim=embedding_dim, batch_size=batch_size):
        
        super(StudyLevelLSTM, self).__init__()

        self.nb_layers = nb_layers
        self.nb_lstm_units = nb_lstm_units
        self.embedding_dim = embedding_dim
        self.batch_size = batch_size

        self.lstm = nn.LSTM(
            input_size=self.embedding_dim,
            hidden_size=self.nb_lstm_units,
            num_layers=self.nb_layers,
            batch_first=True
        )

        # output layer
        self.linear = nn.Linear(self.nb_lstm_units, 9)

    def init_hidden(self):
        # the weights are of the form (nb_layers, batch_size, nb_lstm_units)
        hidden_h0 = torch.randn(self.nb_layers, self.batch_size, self.nb_lstm_units)
        hidden_c0 = torch.randn(self.nb_layers, self.batch_size, self.nb_lstm_units)

        hidden_h0 = hidden_h0.to(device)
        hidden_c0 = hidden_c0.to(device)

        hidden_h0 = Variable(hidden_h0)
        hidden_c0 = Variable(hidden_c0)

        return (hidden_h0.to(device), hidden_c0.to(device))
    
    def forward(self, X):
        # reset the LSTM hidden state. Must be done before you run a new batch. Otherwise the LSTM will treat
        # a new batch as a continuation of a sequence
        self.hidden = self.init_hidden()

        batch_size, seq_len, _ = X.size()

        # pack_padded_sequence so that padded items in the sequence won't be shown to the LSTM
        #X = torch.nn.utils.rnn.pack_padded_sequence(x, X_lengths, batch_first=True)

        # now run through LSTM
        X, self.hidden = self.lstm(X, self.hidden)
        
        X = self.hidden[0]
        X = X.contiguous()
        X = X.view(-1, X.shape[2])

        # run through actual linear layer
        X = self.linear(X)

        # I like to reshape for mental sanity so we're back to (batch_size, seq_len, nb_tags)
        X = X.view(batch_size, 9)

        return X

In [61]:
# sanity test

model = StudyLevelLSTM()
print(model)
device = 'cpu'
model.to(device)
x,y = train_dataset[1]
x = torch.tensor(x)
x = x.unsqueeze(0).float()
x = x.to(device)

pred = model(x)
print(pred)
print(y)

StudyLevelLSTM(
  (lstm): LSTM(2048, 2048, batch_first=True)
  (linear): Linear(in_features=2048, out_features=9, bias=True)
)
tensor([[ 0.0113,  0.0154, -0.0072, -0.0215, -0.0083, -0.0220, -0.0144,  0.0094,
         -0.0046]], grad_fn=<ViewBackward>)
[1. 0. 0. 0. 0. 0. 0. 0. 0.]


In [62]:
bce = torch.nn.BCEWithLogitsLoss()
def criterion(logits, target):
    loss = bce(logits.view(-1), target.view(-1))
    return loss

def train_epoch(model, loader, optimizer):

    model.train()
    train_loss = []
    bar = tqdm(loader)
    for (data, target) in bar:
        data, target = data.to(device), target.to(device)
        
        # zero gradients
        optimizer.zero_grad()

        #forward pass
        logits = model(data.float())
        
        # calculate loss
        loss = criterion(logits, target)

        # backpropagate the loss (backward pass)
        loss.backward()
 
        # update parameters based on accumulated gradients
        optimizer.step()

        loss_np = loss.detach().cpu().numpy()

        train_loss.append(loss_np)
        average_loss = sum(train_loss) / len(train_loss)
        bar.set_description('loss: %.5f, mean: %.5f' % (loss_np, average_loss))

    return float(average_loss)

def valid_epoch(model, loader):
    model.eval()
    val_loss = []
    pos_logits = []
    neg_logits = []

    with torch.no_grad():
        for (data, target) in tqdm(loader):
            data, target = data.to(device), target.to(device)
            logits = model(data.float())

            loss = criterion(logits, target)

            loss_np = loss.detach().cpu().numpy()

            val_loss.append(float(loss_np))

            logits = logits.squeeze()
            target = target.squeeze()
            
            for i in range(logits.shape[-1]):
                b = target[i].detach().cpu().numpy()
                if b == 1:
                    pos_logits.append(float(logits[i].detach().cpu().numpy()))
                else:
                    neg_logits.append(float(logits[i].detach().cpu().numpy()))
                    
    val_loss_mean = sum(val_loss) / len(val_loss)
    
    neg_logits_tensor = torch.FloatTensor(neg_logits).cuda()
    pos_logits_tensor = torch.FloatTensor(pos_logits).cuda()
    
    neg_loss = criterion(neg_logits_tensor, torch.zeros(neg_logits_tensor.shape).float().cuda())
    pos_loss = criterion(pos_logits_tensor, torch.ones(pos_logits_tensor.shape).float().cuda())
    
    neg_loss = neg_loss.detach().cpu().numpy()
    pos_loss = pos_loss.detach().cpu().numpy()
    
    neg_mean = float(torch.sigmoid(neg_logits_tensor).mean().detach().cpu().numpy())
    pos_mean = float(torch.sigmoid(pos_logits_tensor).mean().detach().cpu().numpy())
    
    return float(val_loss_mean), float(pos_loss), float(neg_loss), pos_mean, neg_mean

def get_optimizer(lr, model):
    optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=0.0001)
    return optimizer

init_lr = 0.1
n_epochs = 20
device = 'cuda'

# reduce LR by gamma three times every 10 epochs
# each epoch is 100,000 samples
#gamma = 10
#schedule = [10, 20, 30]

#model = resnext101.to(device)
optimizer = get_optimizer(init_lr, model)
model.to(device)

master_train_loss = []
master_valid_loss = []
epoch = 1
#best_valid_loss = 10


while epoch <= n_epochs:
    print(time.ctime(), 'Epoch:', epoch)
    
    # update learning rate 
    #if epoch in schedule:
    #    new_lr = optimizer.param_groups[0]["lr"] / gamma
     #   optimizer = get_optimizer(new_lr, model)
    
    #train_loader, valid_loader = get_loaders(epoch)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=1,num_workers=1)
    valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=1,num_workers=1)

    # train
    train_loss = train_epoch(model, train_loader, optimizer)
    optimizer.zero_grad()
    
    # validate
    valid_loss, pos_loss, neg_loss, pos_mean, neg_mean = valid_epoch(model, valid_loader)
    #valid_loss = 0
    #pos_loss  = 0
    #neg_loss = 0
    #pos_mean = 0
    #neg_mean = 0
    
    content = time.ctime() + ' ' + f'Epoch {epoch}, lr: {optimizer.param_groups[0]["lr"]:.7f}, train loss: {np.mean(train_loss):.5f}, valid loss: {(valid_loss):.5f}'
    print(content)
    content = f'pos loss: {(pos_loss):.5f}, neg loss: {(neg_loss):.5f}, pos mean: {(pos_mean):.5f}, neg mean {(neg_mean):.5f}'
    print(content)
    master_train_loss.append(train_loss)
    master_valid_loss.append(valid_loss)
    
    # save loss data and model weights
    #with open('train_loss.pkl', 'wb') as f:
        #pickle.dump(master_train_loss, f)
    #with open('valid_loss.pkl', 'wb') as f:
        #pickle.dump(master_valid_loss, f)
        
        #torch.save(model.state_dict(), 'model-resnext-50-{}.pth'.format(epoch))
        #best_valid_loss = valid_loss
    
    epoch += 1
           

  0%|          | 0/2651 [00:00<?, ?it/s]

Wed Dec  2 22:39:18 2020 Epoch: 1


  0%|          | 0/2651 [00:00<?, ?it/s]

Wed Dec  2 22:42:38 2020 Epoch 1, lr: 0.1000000, train loss: 0.33492, valid loss: 0.34182
pos loss: 0.98099, neg loss: 0.21307, pos mean: 0.43019, neg mean 0.17523
Wed Dec  2 22:42:38 2020 Epoch: 2


loss: 0.52156, mean: 0.33556:  35%|███▌      | 932/2651 [00:55<01:42, 16.82it/s]


KeyboardInterrupt: 